In [0]:
sc

Out[1]:

SparkContext 

 Spark UI 

 
 Version 
 v3.1.0 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [0]:
from pyspark.sql.functions import col, desc
from graphframes import GraphFrame
from pyspark.sql import functions as F


In [0]:
# Dataset Link https://snap.stanford.edu/data/soc-redditHyperlinks-body.tsv
# Data containe subreddit-to-subreddit hyperlink network
data = spark.read.csv('/FileStore/tables/soc_redditHyperlinks_body.tsv', sep = '\t', inferSchema=True, header=True)

reddit_data = data.select(['SOURCE_SUBREDDIT', 'TARGET_SUBREDDIT'])
reddit_data = reddit_data.dropna()

In [0]:
reddit_data.show(5)

+----------------+----------------+
SOURCE_SUBREDDIT|TARGET_SUBREDDIT|
+----------------+----------------+
 leagueoflegends| teamredditteams|
 theredlion| soccer|
 inlandempire| bikela|
 nfl| cfb|
 playmygame| gamedev|
+----------------+----------------+
only showing top 5 rows

In [0]:
reddit_data = data.select(['SOURCE_SUBREDDIT', 'TARGET_SUBREDDIT'])
reddit_data = reddit_data.dropna()
s = reddit_data.selectExpr('explode(array(SOURCE_SUBREDDIT, TARGET_SUBREDDIT))').distinct()
s = s.select(col('col').alias('id'))
reddit_data = reddit_data.select(col('SOURCE_SUBREDDIT').alias('src'), col('TARGET_SUBREDDIT').alias('dst'))

graph_d = GraphFrame(v = s, e = reddit_data)

In [0]:
# 

In [0]:
graph_d.outDegrees.sort(desc('outDegree')).show(5)

+---------------+---------+
 id|outDegree|
+---------------+---------+
 subredditdrama| 4665|
 circlebroke| 2358|
shitliberalssay| 1968|
 outoftheloop| 1958|
 copypasta| 1824|
+---------------+---------+
only showing top 5 rows

In [0]:
# subredditdrama has most outDegree which means it contains many links to other subreddits
# same with other subreddits as well

In [0]:
graph_d.inDegrees.sort(desc('inDegree')).show(5)

+--------------+--------+
 id|inDegree|
+--------------+--------+
 askreddit| 7329|
 iama| 3694|
 pics| 2779|
writingprompts| 2490|
 videos| 2446|
+--------------+--------+
only showing top 5 rows

In [0]:
# askreddit has a high incoming degree which means there are many links pointing to r/askreddit on the other subreddits
# same with other subreddits

In [0]:
rank = graph_d.pageRank(maxIter=20)
rank.vertices.orderBy(desc("pagerank")).select("id", "pagerank").show(5)

+---------------+------------------+
 id| pagerank|
+---------------+------------------+
 askreddit| 585.6736429576958|
 iama|476.33018572752746|
 videos|302.86001324046646|
 pics|238.62697263979462|
leagueoflegends| 187.8660309544342|
+---------------+------------------+
only showing top 5 rows

In [0]:
# Page rank gives us askreddit, iama, videos, pics and leagueoflegends as top 5 nodes with highest PageRank values

In [0]:
spark.sparkContext.setCheckpointDir('/FileStore/checkpoints')
comp = graph_d.connectedComponents()
# comp.groupBy('component').agg(F.collect_list('id').alias('subreddits'),F.count('id').alias('total_count')).orderBy(desc('total_count'))

+------------+--------------------+-----------+
 component| subreddits|total_count|
+------------+--------------------+-----------+
 0|[anime, muhfreedo...| 34671|
120259084300|[bitcoinlab, bott...| 42|
 77309411334|[alliterasie, eco...| 9|
180388626464|[courseherohelp, ...| 8|
 94489280512|[0x10c, ixion, tr...| 5|
+------------+--------------------+-----------+
only showing top 5 rows

In [0]:
display(comp.groupBy('component').agg(F.collect_list('id').alias('subreddits'),F.count('id').alias('total_count')).orderBy(desc('total_count')).limit(5))

component subreddits total_count 0 List(anime, muhfreedoms, ck2succession, travel, shotbow, redditisfun, poetry, lojban, jewelry, pervinca, arthistorycirclejerk, cubs, art, bestofbapc, methadone, creepypasta, mysterywriting, mazacoin, crohnsdisease, dolphinswarroom, battlefront, metafeminism, tempus_fugit, left4dead, foreveralonedating, needvocals, pools, ukraina, arumba07, mirrornetworktest, marxism, zombieboxes, kornaycoollair, mastergruntr75, paramedics, isp, marvelheroes, kitchener, lifeguards, enhancement, television, karmacourtattorneys, giftcardexchange, choosetheadventure, vndiscuss, feedthebeastservers, fatpeoplestories, rapbattles, occult, scifi, rfelectronics, nba_draft, easyroleplay, mvtest2, conspiracyhub, askbuddhist, letsmakethisathing, ableton, asianamerican, suchtradebot, iopsychology, treedibles, reddit_reputation, v8supercars, esperanto, shittycosplay, viewjacking, lemansfm, customcovers, crowdfundbitcoin, mafia, myductapesmagicaland2, sneakerdeals, allthingsadc, larp, lolesports, titlegore, finland, mafia_40kimperials, canadian_eclassifieds, maxmsp, tagprowcbosnia, kateeowen, asklaw, appideas, tarnation, exatheist, wallpaperpacks, offmychest, australia, hockey, nottheonion, thehobbit, brokehugs, lucky7s, dnd, freedesign, drunkvapes, goldfish, trfu, stockholm, greece, chemistry, photoshoprequest, contentstrategist, mh370, lorde, nerf, fixit, interiordesign, growmybusiness, powerpoint, chennai, australiancattledog, dataisugly, magiccardpulls, publicfreakout, translatednews, meraki, dogecoingifs, antimemewatch, maydaypac, 48hourcomic, letswriteadirtystory, teamredditteams, corejerk, bretagne, oakland, bestoftypos, worldevents, footballdownload, redditracing, whatisthis, kzoo, denvergamers, atlantathrashers, proassad, portugal, falloutlore, readitnow, paranormalhelp, topcoin, megalinks, mustachianism, vinhaven, biwomen, providence, highschooldxd, daedricgod101sarchive, deadredditors, ratrod, secrettoeveryone, auroraco, watchingcongress, dinovanilla, film, fairbanks, boxeo, hcfwfa, androidhelp, cardwarstcg, kratom, conspiratard, futurology, nosleep, leagueoflegendsmeta, footballhighlights, grandtheftautov, hamcraft, israel, makeupaddiction, hackintosh, friendlyreferrals, ploungemafiaghosties, ukraine, bostonbruins, aleague, smartrecovery, lgbteens, dogestarter, holmes, iosprogramming, hailcorporategold, subdreddit, girlsgonebitcoin, lolurf, debate, campuscarry, voxels, retrogaming, cswchunk, physicsjokes, musicguides, universeproject, dcss, aboutreddit, boardgamerecommender, focust25, johndotcom, inven, thoughtfactory, laliga, shitstatistssay, loseit, starcitizencss, wargaming, whatsthisworth, gonewildaudio, halflife, nsfwasmr, ps3, beercirclejerk, designporn, nova, transgendercirclejerk, cfbcirclejerk, americanhorrorstory, oceanicdota, trgartists, drugslist, ukgames, mcwt, idiotsfightingthings, thegabber, battlefield, peanuts, funbelt, smartereveryday, anarchychess, reddcointips, bf4_x1, chickens, karlstad, diybeauty, njriders, nsfr, integra, redditalchemy, elonmusk, sousvide, uae, socialjustice, adagency, trtest, spacewalls, 350z, uofcmm, mandolin, cardinals, ecouteca, astronomy, respectporn, ncbeer, stocks, antm, awwducational, instagc, spammeddomains, gta, anarchy101, callofduty, tokyo, greenlightquality, lingerie, longdistancemc, sfbayrunners, banishedmodding, stopcomcast, matthewg, playmindcrack, dogecoin_pr, hearthstonevods, hardcoresmp, ploompax, kevin, selfharmcommunity, thegirlsurvivalguide, asatru, instagram, fictionbrawl, leagueofquests, forwardsfromgrandma, d3loot, tpgflairs, thefulmercup, adviceatheists, naslsoccer, redditcantjump, oily, designjobs, delhi, eritrea, musicprogramming, pokemonturfwars, gearsofwar, online2k14, adamcarolla, hhsrocketry, girlsundpanzer, seriousdoge, uhccourtroom, army, catholicism, gaeilge, houstonsoccer, space, suggestalaptop, huntsvillealabama, beertrade, viddme, alabama, letsplayvideos, edc, jewboyconspiracy, iphone, torontojobs, southbend, borussiadortmund, buttcoin,

In [0]:
# We can see that there are many subreddits connected together (since commenting/linking other subreddits on a subreddit is very random) hence in we get one group having 34K values
# As we can see that on the 4th highest there are many subreddits related to Coursehero 

In [0]:
tria = graph_d.triangleCount()
tria.select("id", "count").orderBy(desc('count')).show(5)

+--------------+-----+
 id|count|
+--------------+-----+
 askreddit|31967|
subredditdrama|26072|
 iama|24581|
 outoftheloop|15898|
 videos|11938|
+--------------+-----+
only showing top 5 rows

In [0]:
# askreddit has the most traingles count followed by subredditdrama, 
# iama ( I am a _____, Ask Me Anything) one of the popular subreddit
# outoftheloop (A subreddit to help you keep up to date with what's going on with reddit and other stuff.), 
# videos (people sharing videos of variety) highly visited and has high subscribers


# The results of the above analysis are quite expected since the subreddits are most visited. Some of them are still most visited till today. 
# The data is old and does not contain new and popular subreddits like memes etc. 